# Db2 JSON Function Overview

## Db2 JSON Functions
Db2 Version 11.1 Fix pack 4 introduced a subset of the JSON SQL functions defined by ISO and that set is shown in the table below.

| Function | Description |
|:---------|:------------|
| `BSON_TO_JSON` | Convert BSON formatted document into JSON strings 
| `JSON_TO_BSON` | Convert JSON strings into a BSON document format 
| `JSON_ARRAY` | Creates a JSON array from input key value pairs
| `JSON_OBJECT` | Creates a JSON object from input key value pairs 
| `JSON_VALUE` | Extract an SQL scalar value from a JSON object 
| `JSON_QUERY` | Extract a JSON object from a JSON object 
| `JSON_TABLE` | Creates a SQL table from a JSON object 
| `JSON_EXISTS` | Determines whether a JSON object contains the desired JSON value

These functions are all part of the SYSIBM schema, so a user does not require permissions in order to use them for development or general usage. The functions can be categorized into three broad categories:
#### Conversion functions
The `BSON_TO_JSON` and `JSON_TO_BSON` functions are used to convert JSON character data into the binary BSON format and vice-versa. Conversion functions are optional and are discussed in the section below. These functions are not actually part of the ISO specifications and are provided simply for your convenience.
#### Retrieval functions
The `JSON_VALUE` and `JSON_QUERY` functions are used to retrieve portions of a document as SQL or JSON scalar values, while `JSON_TABLE` can be used to format JSON documents into a table of rows and columns. The `JSON_EXISTS` function can be used in conjunction with the retrieval functions to check for the existence of a field.
#### Publishing Routines
The `JSON_ARRAY` and `JSON_OBJECT` functions are used to create JSON objects from relational data.

### Common Db2 JSON Parameters
A majority of the Db2 ISO JSON functions depend on two parameters that are supplied at the beginning of a function. These parameters are: 
* JSON Expression
* JSON Path Expression

#### JSON Expression
The JSON expression refers to either a column name in a table where the JSON document is stored (either in JSON or BSON format), a JSON or BSON literal string, or a SQL variable containing a JSON or BSON string. 

The examples below illustrate these options.
* A column name within a Table
```
JSON_VALUE(CUSTOMER.JSON_DOC,…)
```
* Using a character string as the argument
```
JSON_VALUE('{"first":"Thomas","last":"Hronis":}',…)
```

* Using an SQL variable
```
CREATE VARIABLE EXPR VARCHAR(256) DEFAULT('{"first":"Thomas"}')
JSON_VALUE(EXPR,…)
```

The JSON expression can also include a modifier of `FORMAT JSON` or `FORMAT BSON`. The `FORMAT` clause is optional and by default the Db2 functions use the data type of the supplied value to determine how to interpret the contents. In the event that you need to override how the JSON field is interpreted, you must use the `FORMAT` option.

#### JSON Path Expression
A JSON path expression is used to navigate to individual values, objects, arrays, or allow for multiple matches within a JSON document. The JSON path expression is based on the syntax that is fully described in the notebook on  JSON Path Expressions. 
The following list gives a summary of how a path expression is created but the details of how the matches occur are documented in the next chapter.
* The top of any path expression is the anchor symbol (`$`)
* Traverse to specific objects at different levels by using the dot operator (`.`)
* Use square brackets `[]` to refer to items in an array with the first item starting at position zero (i.e. first element in an array is accessed as `arrayname[0]`)
* Use the backslash `\` as an escape character when key names include any of the JSON path characters `(.,*,$,[,])`
* Use the asterisk (`*`) to match any object at the current level
* Use the asterisk (`*`) to match all objects in an array or retrieve only the value fields from an object

The path expression can have an optional name represented by the `AS path-name` clause. The `AS` clause is included for compatibility with the ISO SQL standard but currently does not have any effect on the Db2 JSON functions.

## Sample JSON Functions
The following SQL demonstrates some of the JSON functions that are available in Db2. If you require more detailed explanation of the Db2 JSON functions refer to the [Db2 JSON ebook](./media/Db2V11-JSON-ebook.pdf) available on this system.

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run db2.ipynb
%run connection.ipynb

This statement will create a variable named __`customer`__ which will be used for some of the examples.

In [ ]:
customer = {
    "customerid": 100000,
    "identity": 
      {
        "firstname": "Jacob",
        "lastname": "Hines",
        "birthdate": "1982-09-18"
      },
    "contact": 
      {
        "street": "Main Street North",
        "city": "Amherst",
        "state": "OH",
        "zipcode": "44001",
        "email": "Ja.Hines@yahii.com",
        "phone": "813-689-8309"
      },
    "payment": 
      {
        "card_type": "MCCD",
        "card_no": "4742-3005-2829-9227"
      },
    "purchases": 
      [
        {
          "tx_date": "2018-02-14",
          "tx_no": 157972,
          "product_id": 1860,
          "product": "Ugliest Snow Blower",
          "quantity": 1,
          "item_cost": 51.86
        }
      ]
}

### JSON_EXISTS
`JSON_EXISTS` allows you to check whether or not a valid JSON key exists within a document for the provided search path. You can use the result of this function to determine if the contents of a JSON document are consistent with your expectations and to decide whether or not to take further action or retrieve the value. You can also use this function to validate that the JSON document is properly formed.

Check to see if the customer made a __`purchase`__.

In [ ]:
%sql VALUES JSON_EXISTS(:customer,'$.purchases')

### JSON_VALUE
The `JSON_VALUE` function is used to retrieve a single value from a JSON document in the form of a "native" SQL data type which can be directly referenced by a user application like any other SQL data value or it can be embedded within another SQL statement. 

This function implicitly converts the returning value from its original JSON format to the identified Db2 data type. Since it is a scalar function, `JSON_VALUE` will only return a single value and, if the evaluation of the JSON path expression results multiple JSON values being returned, `JSON_VALUE` will return an error.

This SQL will retrieve the __`customerid`__ field.

In [ ]:
%sql VALUES JSON_VALUE(:customer,'$.customerid')

### JSON_QUERY
Because `JSON_VALUE` is a scalar function which is returning values using native Db2 data types, it is limited to retrieving atomic or individual values from within a document. In order to extract native JSON values, which can include complex ones such as multiple array values or entire JSON objects, you must use the `JSON_QUERY` function. The `JSON_QUERY` function has a similar syntax as `JSON_VALUE` but adds some modifiers to handle complex results such as arrays. 

This function implicitly returns values in their original JSON or BSON format. Since it is a scalar function, `JSON_QUERY` can only return a single JSON value as its result; if the evaluation of the JSON path expression results multiple, independent JSON values being returned, `JSON_QUERY` will process this as an error.

This SQL will retrieve an entire object called __`identity`__ rather than an individual value.

In [ ]:
%sql -j VALUES JSON_QUERY(:customer,'$.identity')

### JSON_TABLE
Up to this point, the JSON functions can be used to check for the existence of an object and retrieve individual values. While these functions can be used to retrieve all of the values within a JSON document by using multiple calls, an easier method exists in the form of the new `JSON_TABLE` function based on the ISO SQL standard. While this function does not yet implement all of the ISO `JSON_TABLE` function definition, the part that has been implemented in Db2 is still very useful and can help simplify things for you. 

The `JSON_TABLE` function provides two ways to define a column. Regular column expressions mimic the `JSON_VALUE` function, while formatted column expressions use features from the `JSON_QUERY` function. You can have different column definitions in the same `JSON_TABLE` invocation.

This example will retrieve all of the __`personal`__ information as a table.

In [ ]:
%%sql
WITH CUSTOMER(INFO) AS (VALUES :customer)
SELECT T.* FROM CUSTOMER, 
  JSON_TABLE(INFO, 'strict $'
    COLUMNS( 
      FIRST_NAME VARCHAR(20) PATH '$.identity.firstname',
      LAST_NAME  VARCHAR(20) PATH '$.identity.lastname',
      BIRTHDATE  DATE        PATH '$.identity.birthdate')
    ERROR ON ERROR) AS T;

### JSON_OBECT
Up to this point, the JSON functions allow you to extract values, objects, and arrays from documents. There are many circumstances where you want to be able to take the existing data in a table and make it available to outside world as JSON data. Publishing data as JSON is particularly useful when sending results back to an application that expects JSON.

Db2 provides two of the ISO SQL JSON publishing functions: `JSON_ARRAY` and `JSON_OBJECT`. The combination of these two functions can provide a way of generating most JSON documents.

The `JSON_OBJECT` function will generate a JSON object by creating `key:value` pairs. Using the customer table defined in the `WITH` statement, we can generate a JSON object that contains all of the customers information as a JSON document using the `JSON_OBJECT` function.

This example will publish one record as a JSON object.

In [ ]:
%%sql -j
WITH CUSTOMER(CUSTNO, FIRSTNAME, LASTNAME, BIRTHDATE, INCOME) AS
  (
  VALUES
     (1, 'Thomas', 'Hronis', '1999-01-01', 50000)
  )
SELECT 
   JSON_OBJECT ( 
               KEY 'customer' VALUE JSON_OBJECT
                   ( 
                   KEY 'id' VALUE CUSTNO,
                   KEY 'name' VALUE JSON_OBJECT
                       (
                       KEY 'first' VALUE FIRSTNAME,
                       KEY 'last'  VALUE LASTNAME
                       ) FORMAT JSON,
                   KEY 'birthdate' VALUE BIRTHDATE,
                   KEY 'income'    VALUE INCOME
                   ) FORMAT JSON
               )
FROM CUSTOMER

### JSON_ARRAY
`JSON_OBJECT` is able to create complex JSON documents from data within a table, but it is not able to generate arrays. In order to create arrays, we must use the `JSON_ARRAY` function. There are two forms of the `JSON_ARRAY` function. The first version is similar to the `JSON_OBJECT` function where you supply a list of values to create an object. There is no key associated with a JSON array, so you only need to supply the list of values that you want in there.

This SQL will create an array from a list of values. Note that the data types do not need to be the same in a JSON array!

In [ ]:
%sql -j  VALUES JSON_ARRAY( 1523, 902, 'Thomas', 7777);

While the `JSON_ARRAY` function can be used by itself, it produces a JSON array value not a valid JSON object. The output from this function is meant to be used as part of a `JSON_OBJECT` structure.

The second form of the `JSON_ARRAY` function uses the results of a SQL select statement to build the array values.
Only one `SELECT` statement can be used in the body of the function – you cannot have multiple `SELECT` commands in a list! If you do need to create an array from multiple sources, you should look at using a `SELECT` statement with `UNION` to create one list of items.

The following example publishes all of the department numbers for the departments that start with the letter `B`.

In [ ]:
%%sql -j
VALUES JSON_OBJECT(
                   KEY 'departments' VALUE 
                       JSON_ARRAY(SELECT DEPTNO FROM DEPARTMENT 
                                         WHERE DEPTNAME LIKE 'B%') 
                   FORMAT JSON
                  );

### Summary
Db2 supports a number of the new ISO JSON functions directly in the database. Users can use this new syntax to store, query, and publish JSON data from within Db2.

#### Copyright (C) IBM 2021, George Baklarz [baklarz@ca.ibm.com]